In [51]:
import pandas as pd
import numpy as np
import networkx as nx
import os
from os import listdir
import random

# Salmonella enterica subsp. enterica serovar Typhimurium str. LT2

In order to have the same RefSeq on both databases, this subspecie is used

In [109]:
bacteria = 'Salmonella'
root = 'E:/User/bruna.fistarol/Documents/GitHub'

# Data From PATRIC

## Genomic Features

The table below contains a list of genomic features, including coding DNA.

Each feature is solely identified by BRC ID and associated to a protein family referred as PATRIC genus-specific families (PLfams). Most of the genes has the associated RefSeq.

In [53]:
features = pd.read_csv('genome_features.csv')

Through this table, we extract useful data to map protein families referred by Nguyen et. al.:

In [54]:
plf = features[['BRC ID', 'PATRIC genus-specific families (PLfams)']].astype("string")
plf.columns = ['BRC_ID', 'PLFam']
plf.set_index('BRC_ID', inplace = True)
plf

,PLFam
BRC_ID,
fig|99287.12.peg.977,PLF_590_00000012
fig|99287.12.peg.978,PLF_590_00014201
fig|99287.12.peg.979,PLF_590_00019426
fig|99287.12.peg.980,PLF_590_00015851
fig|99287.12.peg.981,PLF_590_00000025
...,...
fig|99287.12.peg.4975,PLF_590_00006047
fig|99287.12.peg.4976,PLF_590_00005031
fig|99287.12.peg.4844,PLF_590_00005081


## Specialty Genes

The table containing specialty genes relates several genomic features to a relevant property. Here the table is filtered by the antibiotic resistance property

In [55]:
AMR_genes = pd.DataFrame(pd.read_csv('specialty_genes.csv')['BRC ID'])
AMR_genes.columns = ['BRC_ID']

## Data from Nguyen et. al.

In [56]:
plf_500 = []

datadir = f'{root}/Nguyen_et_al_2020/{bacteria}/fasta.500.0'
for strain in listdir(datadir):
    with open(os.path.join(datadir, strain), 'r') as sequences:
        for line in sequences:
            if line[0] == '>':
                plf_500.append(line[1:len(line)-1])
                
plf_500 = pd.DataFrame(np.unique(plf_500))
plf_500.columns = ['Protein Family ID']

## RefSeq Mapping

In [57]:
refseq = features[['BRC ID', 'RefSeq Locus Tag']]
refseq.columns = ['BRC_ID', 'RefSeq']
refseq.set_index('BRC_ID', inplace = True)

In [58]:
AMR_refseq = features[['RefSeq Locus Tag']][features['BRC ID'].isin(AMR_genes['BRC_ID'])].reset_index(drop = True)
AMR_refseq.columns = ['AMR_RefSeq']

In [59]:
plf_map_refseq = features[['RefSeq Locus Tag', 'PATRIC genus-specific families (PLfams)']][features['PATRIC genus-specific families (PLfams)'].isin(plf_500['Protein Family ID'])].reset_index(drop = True)
plf_map_refseq.columns = ['RefSeq', 'PLF']
plf_map_refseq.dropna(inplace = True)
plf_map_refseq = plf_map_refseq.reset_index(drop = True)

In [60]:
plf_map_refseq

,RefSeq,PLF
0,STM0934,PLF_590_00001917
1,STM0936,PLF_590_00003070
2,STM0942,PLF_590_00000215
3,STM0961,PLF_590_00001933
4,STM0969,PLF_590_00001385
...,...,...
487,STM0869,PLF_590_00002654
488,STM0874,PLF_590_00000196
489,STM0876,PLF_590_00001271
490,STM0889,PLF_590_00000747


## Protein Interaction Network

In [61]:
ppi_patric = pd.read_csv('ppi_patric.csv')
ppi_patric = ppi_patric[['Interactor A ID', 'Interactor B ID']].astype("string")
ppi_patric.columns = ['Interactor_A_ID', 'Interactor_B_ID']

In [62]:
ppi_refseq = ppi_patric
for i in range(len(ppi_refseq['Interactor_A_ID'])):
    if ppi_refseq['Interactor_A_ID'][i] in list(refseq.index):
        ppi_refseq.at[i, 'Interactor_A_ID'] = refseq.loc[ppi_refseq['Interactor_A_ID'][i]].RefSeq
    else:
        ppi_refseq.drop(inplace = True, labels = i)
        
ppi_refseq.reset_index(inplace = True, drop = True)
        
for i in range(len(ppi_refseq['Interactor_B_ID'])):
    if ppi_refseq['Interactor_B_ID'][i]in (refseq.index):
        ppi_refseq.at[i, 'Interactor_B_ID'] = refseq.loc[ppi_refseq['Interactor_B_ID'][i]].RefSeq
    else:
        ppi_refseq.drop(inplace = True, labels = i)

In [63]:
ppi_string = pd.read_csv('ppi_string.txt', sep = ' ')
ppi_string.columns = ['Interactor_A_ID', 'Interactor_B_ID', 'weight']
ppi_string.replace('99287.', '', regex = True, inplace = True)

In [70]:
ppi_all = pd.DataFrame(pd.concat([ppi_refseq, ppi_string], axis = 0))

In [71]:
ppi_all.fillna(999, inplace = True)

In [72]:
ppi_all

,Interactor_A_ID,Interactor_B_ID,weight
0,STM2168,STM2439,999.0
1,STM0264,STM0798,999.0
2,STM0558,STM1340,999.0
3,STM2999,STM3000,999.0
4,STM1398,STM1396,999.0
...,...,...,...
105069,STM4585,STM4578,403.0
105070,STM4588,STM4589,627.0
105071,STM4589,STM4588,627.0
105072,STM4592,STM4593,173.0


In [78]:
ppi = ppi_all[ppi_all['weight'] > 600].reset_index()

In [82]:
conserved_ppi_A = plf_map_refseq[plf_map_refseq['RefSeq'].isin(ppi['Interactor_A_ID'])]['RefSeq']
conserved_ppi_B = plf_map_refseq[plf_map_refseq['RefSeq'].isin(ppi['Interactor_B_ID'])]['RefSeq']

conserved_ppi = pd.DataFrame(pd.concat([conserved_ppi_A, conserved_ppi_B], axis = 0).drop_duplicates())

In [83]:
conserved_ppi

,RefSeq
0,STM0934
1,STM0936
2,STM0942
3,STM0961
5,STM0970
...,...
334,STM3686
379,STM4082
385,STM4127
423,STM4381


In [84]:
AMR_ppi_A = AMR_refseq[AMR_refseq['AMR_RefSeq'].isin(ppi['Interactor_A_ID'])]['AMR_RefSeq']
AMR_ppi_B = AMR_refseq[AMR_refseq['AMR_RefSeq'].isin(ppi['Interactor_B_ID'])]['AMR_RefSeq']

AMR_ppi = pd.DataFrame(pd.concat([AMR_ppi_A, AMR_ppi_B], axis = 0))
AMR_ppi.reset_index(drop=True, inplace=True)

In [85]:
AMR_ppi

,AMR_RefSeq
0,STM0087
1,STM0941
2,STM0942
3,STM1197
4,STM0130
...,...
116,STM4338
117,STM4517
118,STM0475
119,STM0476


In [86]:
ppi_info = pd.DataFrame(columns = ['Conserved Gene', 'Shortest Path to an AMR gene (length)',])

ppi_info['Conserved Gene'] = conserved_ppi.reset_index(drop = True)['RefSeq']

In [96]:
ppi_graph = nx.from_pandas_edgelist(ppi, 'Interactor_A_ID', 'Interactor_B_ID', edge_attr = ['weight'])

In [97]:
ppi_graph.number_of_edges()

18347

In [98]:
ppi_graph.number_of_nodes()

3445

In [99]:
i = conserved_ppi['RefSeq'][0]
j = AMR_ppi['AMR_RefSeq'][0]
nx.has_path(ppi_graph, i, j)

True

In [101]:
nx.shortest_path(ppi_graph, i, j)

['STM0934', 'STM2555', 'STM0087']

In [90]:
idx = 0
for i in conserved_ppi['RefSeq']:
    lengths = []
    for j in AMR_ppi['AMR_RefSeq']:
        if nx.has_path(ppi_graph, i, j):
            lengths.append(nx.shortest_path_length(ppi_graph, i, j))
    if lengths:        
        ppi_info['Shortest Path to an AMR gene (length)'][idx] = min(lengths)
        
    idx += 1

In [91]:
print(ppi_info.groupby(['Shortest Path to an AMR gene (length)']).size().reset_index(name='Count'))

   Shortest Path to an AMR gene (length)  Count
0                                      0     10
1                                      1     55
2                                      2    165
3                                      3    134
4                                      4     33
5                                      5      6
6                                      6      1


In [93]:
list(set(AMR_refseq['AMR_RefSeq']) & set(plf_map_refseq['RefSeq']))

['STM4292',
 'STM0046',
 'STM3441',
 'STM3186',
 'STM2272',
 'STM3390',
 'STM2814',
 'STM1700',
 'STM3873.S',
 'STM4293',
 'STM0942']

In [102]:
plf_map_refseq = plf_map_refseq.set_index('RefSeq')

In [103]:
for i in ppi_info['Shortest Path to an AMR gene (length)'].unique():
    gene_set = ppi_info[ppi_info['Shortest Path to an AMR gene (length)'] == i]['Conserved Gene']
    globals()[f'plf_length_{i}'] = plf_map_refseq.loc[list(gene_set)]['PLF']

In [105]:
bacdir = f'{root}/Fistarol_2022_2.0/{bacteria}'

os.mkdir(bacdir)

for i in ppi_info['Shortest Path to an AMR gene (length)'].unique():
    newdir = f'length.{i}'
    lendir = os.path.join(bacdir, newdir)
    os.mkdir(path)
    
    sample = f'{root}/Nguyen_et_al_2020/{bacteria}/fasta.500.0'
    for strain in listdir(sample):
        with open(os.path.join(lendir, strain), 'a') as mystrain:
            with open(os.path.join(sample, strain), 'r') as sequences:
                first_loop = True
                for line in sequences:
                    if line[0] == '>':
                        if first_loop:
                            plfam = line[1:len(line)-1]
                            seq = ''
                            first_loop = False
                            continue
                        if plfam in list((globals()[f'plf_length_{i}'])):
                            mystrain.write('>' + plfam + '\n')
                            mystrain.write(seq)
                        plfam = line[1:len(line)-1]
                        seq = ''
                    else:
                        seq += line
                if plfam in list((globals()[f'plf_length_{i}'])):
                            mystrain.write('>' + plfam + '\n')
                            mystrain.write(seq)

At this point, it is possible to use this new configuration of data to run the model. We can take 25 genes to each strain for lengths equals to 1, 2 and 3, because the results from the paper also are derived from groups of 25 genes, hence, we can compare these results.

In [112]:
dir_25genes = os.path.join(bacdir, '25genes')
os.mkdir(dir_25genes)

for j in [0, 1, 2, 3, 4]:
    
    rand_idx = [sorted(random.sample(range(55), 25)), 
                sorted(random.sample(range(165), 25)), 
                sorted(random.sample(range(134), 25)),
                sorted(random.sample(range(33), 25))]
    
    for i in range(1, len(rand_idx)+1):   

        lendir = os.path.join(bacdir, f'length.{i}')
        repdir = os.path.join(dir_25genes, f'length.{i}.{j}')
        os.mkdir(repdir)

        for strain in listdir(lendir):
            with open(os.path.join(repdir, strain), 'a') as mystrain:
                with open(os.path.join(lendir, strain), 'r') as sequences:
                    c = 0
                    first_loop = True
                    for line in sequences:
                        if line[0] == '>':
                            if first_loop:
                                plfam = line
                                seq = ''
                                c += 1
                                first_loop = False
                                continue
                            if c in rand_idx[i-1]:
                                mystrain.write(plfam)
                                mystrain.write(seq)
                            plfam = line
                            seq = ''
                            c += 1
                        else:
                            seq += line
                    if c in rand_idx[i-1]:
                            mystrain.write(plfam)
                            mystrain.write(seq)

In [113]:
rand_idx = range(10)

lendir = os.path.join(bacdir, 'length.0')
repdir = os.path.join(dir_25genes, 'length.0.0')
os.mkdir(repdir)

for strain in listdir(lendir):
    with open(os.path.join(repdir, strain), 'a') as mystrain:
        with open(os.path.join(lendir, strain), 'r') as sequences:
            c = 0
            first_loop = True
            for line in sequences:
                if line[0] == '>':
                    if first_loop:
                        plfam = line
                        seq = ''
                        c += 1
                        first_loop = False
                        continue
                    if c in rand_idx:
                        mystrain.write(plfam)
                        mystrain.write(seq)
                    plfam = line
                    seq = ''
                    c += 1
                else:
                    seq += line
            if c in rand_idx:
                    mystrain.write(plfam)
                    mystrain.write(seq)